In [ ]:
import geopandas as gpd
import pygeoops
from shapely.geometry import Polygon, MultiPolygon
import numpy as np

## Find the centerline without densifying the polyline/polygons

In [ ]:
# Define the input and output paths
input_road_polygon = "Road.shp"
output_centerline = "Road_Centerline.shp"

try:
    # Load the polygon representing the road contour as a GeoDataFrame
    gdf = gpd.read_file(input_road_polygon)
    print("Polygon data read successfully.")

    # Calculate centerline of the polygons
    gdf.geometry = gdf.geometry.apply(pygeoops.centerline)
    print("Centerline calculated successfully.")

    # Save centerlines in a new shapefile
    gdf.to_file(output_centerline)
    print(f"Centerline saved to {output_centerline}")

except Exception as e:
    print(f"An error occurred: {e}")

## Find the centerline by densifying the polyline/polygons

In [ ]:
# Define the input and output paths
input_road_polygon = "Road.shp"
output_centerline = "Road_Centerline_Densed.shp"

def densify_polygon(polygon, distance):
    """
    Densify the polygon by adding points along its edges.
    """
    if polygon.is_empty:
        return polygon

    # Get the exterior coordinates of the polygon
    exterior_coords = np.array(polygon.exterior.coords)
    
    # Calculate the number of points to add along each edge
    num_points = int(np.ceil(polygon.length / distance))
    
    # Create a new list of coordinates with additional points
    new_coords = []
    for i in range(len(exterior_coords) - 1):
        start = exterior_coords[i]
        end = exterior_coords[i + 1]
        new_coords.append(start)
        for j in range(1, num_points):
            new_coords.append(start + (end - start) * j / num_points)
    new_coords.append(exterior_coords[-1])
    
    # Create a new polygon with the densified coordinates
    densified_polygon = Polygon(new_coords)
    
    return densified_polygon

def densify_geometry(geometry, distance):
    """
    Densify the geometry, handling both Polygon and MultiPolygon geometries.
    """
    if isinstance(geometry, Polygon):
        return densify_polygon(geometry, distance)
    elif isinstance(geometry, MultiPolygon):
        return MultiPolygon([densify_polygon(p, distance) for p in geometry.geoms])
    else:
        return geometry

try:
    # Load the polygon representing the road contour as a GeoDataFrame
    gdf = gpd.read_file(input_road_polygon)
    print("Polygon data read successfully.")

    # Densify the polygons
    gdf.geometry = gdf.geometry.apply(lambda geom: densify_geometry(geom, distance=10.0))
    print("Polygons densified successfully.")

    # Calculate centerline of the polygons
    gdf.geometry = gdf.geometry.apply(pygeoops.centerline)
    print("Centerline calculated successfully.")

    # Save centerlines in a new shapefile
    gdf.to_file(output_centerline)
    print(f"Centerline saved to {output_centerline}")

except Exception as e:
    print(f"An error occurred: {e}")

Polygon data read successfully.
Polygons densified successfully.
